## Проект - Базовый SQL

### Задание №1 

Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(*) AS closed_companies_count
FROM company
WHERE status = 'closed';

### Задание №2 

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total`.

In [ ]:
SELECT 
    --name AS company_name,
    --category_code,
    funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC;

### Задание №3

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с `2011` по `2013` год включительно.

In [ ]:
SELECT
    SUM(price_amount) AS total_cash_deals
FROM acquisition
WHERE term_code = 'cash' AND acquired_at >= '2011-01-01' AND acquired_at <= '2013-12-31';

### Задание №4

Отобразите имя, фамилию и названия аккаунтов людей в поле **network_username**, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT
    first_name,
    last_name,
    twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

### Задание №5

Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле `network_username` содержат подстроку `money`, а фамилия начинается на `K`.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%' AND last_name ILIKE 'K%';

### Задание №6

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code, SUM(funding_total) AS total_investment
FROM company
GROUP BY country_code
ORDER BY total_investment DESC;

### Задание №7

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.

Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at AS round_date,
       MIN(raised_amount) AS min_investment,
       MAX(raised_amount) AS max_investment
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0 AND MIN(raised_amount) <> MAX(raised_amount);

### Задание №8

Создайте поле с категориями:
 - Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
 - Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
 - Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.
Отобразите все поля таблицы `fund` и новое поле с категориями.

In [ ]:
SELECT id,
       name,
       founded_at,
       domain,
       twitter_username,
       country_code,
       investment_rounds,
       invested_companies,
       milestones,
       created_at,
       updated_at,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity_category
FROM fund;

### Задание №9

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT activity_category,
       ROUND(AVG(investment_rounds)) AS avg_investment_rounds
FROM (
    SELECT id,
           name,
           founded_at,
           domain,
           twitter_username,
           country_code,
           investment_rounds,
           invested_companies,
           milestones,
           created_at,
           updated_at,
           CASE
               WHEN invested_companies >= 100 THEN 'high_activity'
               WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
               ELSE 'low_activity'
           END AS activity_category
    FROM fund
) AS categorized_funds
GROUP BY activity_category
ORDER BY avg_investment_rounds;

### Задание №10

Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю.

Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT
    f.country_code,
    --COUNT(f.invested_companies) AS total_invested_companies,
    MIN(f.invested_companies) AS min_invested_companies,
    MAX(f.invested_companies) AS max_invested_companies,
    AVG(f.invested_companies) AS avg_invested_companies
FROM
    fund f
WHERE
    f.founded_at BETWEEN '2010-01-01' AND '2012-12-31'
GROUP BY
    f.country_code
HAVING
    MIN(f.invested_companies) > 0
ORDER BY
    avg_invested_companies DESC,
    country_code ASC
LIMIT 10;

### Задание №11

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT
    p.first_name,
    p.last_name,
    e.instituition AS education_institution
FROM
    people p
LEFT JOIN
    education e ON p.id = e.person_id;

### Задание №12

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT
    c.name AS company_name,
    COUNT(DISTINCT e.instituition) AS num_unique_education
FROM
    company c
LEFT JOIN
    people p ON c.id = p.company_id
LEFT JOIN
    education e ON p.id = e.person_id
GROUP BY
    c.id, c.name
ORDER BY
    num_unique_education DESC
LIMIT 5;

### Задание №13

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT
    c.name
FROM
    company c
JOIN (
    SELECT
        fr.company_id
    FROM
        funding_round fr
    JOIN (
        SELECT
            company_id,
            MIN(funded_at) AS first_funding_date,
            MAX(funded_at) AS last_funding_date
        FROM
            funding_round
        GROUP BY
            company_id
    ) first_last_dates ON fr.company_id = first_last_dates.company_id
    WHERE
        fr.funded_at = first_last_dates.first_funding_date
        AND fr.funded_at = first_last_dates.last_funding_date
) first_last_rounds ON c.id = first_last_rounds.company_id
WHERE
    c.status = 'closed';

### Задание №14

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT
    p.id AS employee_id
FROM
    company c
JOIN (
    SELECT
        fr.company_id
    FROM
        funding_round fr
    JOIN (
        SELECT
            company_id,
            MIN(funded_at) AS first_funding_date,
            MAX(funded_at) AS last_funding_date
        FROM
            funding_round
        GROUP BY
            company_id
    ) first_last_dates ON fr.company_id = first_last_dates.company_id
    WHERE
        fr.funded_at = first_last_dates.first_funding_date
        AND fr.funded_at = first_last_dates.last_funding_date
) first_last_rounds ON c.id = first_last_rounds.company_id
JOIN
    people p ON c.id = p.company_id
WHERE
    c.status = 'closed';

### Задание №15

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT
    p.id AS employee_id,
    e.instituition AS education_institution
FROM
    company c
JOIN (
    SELECT
        fr.company_id
    FROM
        funding_round fr
    JOIN (
        SELECT
            company_id,
            MIN(funded_at) AS first_funding_date,
            MAX(funded_at) AS last_funding_date
        FROM
            funding_round
        GROUP BY
            company_id
    ) first_last_dates ON fr.company_id = first_last_dates.company_id
    WHERE
        fr.funded_at = first_last_dates.first_funding_date
        AND fr.funded_at = first_last_dates.last_funding_date
) first_last_rounds ON c.id = first_last_rounds.company_id
JOIN
    people p ON c.id = p.company_id
JOIN
    education e ON p.id = e.person_id
WHERE
    c.status = 'closed';

### Задание №16

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT ppl.id, COUNT(ed.instituition)
FROM people AS ppl
JOIN education AS ed ON ppl.id = ed.person_id
WHERE ppl.company_id IN (
    SELECT co.id
    FROM company AS co
    JOIN funding_round AS fr ON co.id = fr.company_id
    WHERE co.status = 'closed' AND fr.is_first_round = 1 AND fr.is_last_round = 1
)
GROUP BY 1
HAVING COUNT(DISTINCT ed.instituition) > 0;

### Задание №17

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(num_education_institutions) AS average_education_institutions
FROM (
    SELECT COUNT(ed.instituition) AS num_education_institutions
    FROM people AS ppl
    JOIN education AS ed ON ppl.id = ed.person_id
    WHERE ppl.company_id IN (
        SELECT co.id
        FROM company AS co
        JOIN funding_round AS fr ON co.id = fr.company_id
        WHERE co.status = 'closed' AND fr.is_first_round = 1 AND fr.is_last_round = 1
    )
    GROUP BY ppl.id
    HAVING COUNT(DISTINCT ed.instituition) > 0
) subquery;

### Задание №18

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
SELECT AVG(num_education_institutions) AS average_education_institutions
FROM (
    SELECT COUNT(ed.instituition) AS num_education_institutions
    FROM people AS ppl
    JOIN education AS ed ON ppl.id = ed.person_id
    WHERE ppl.company_id IN (
        SELECT id
        FROM company
        WHERE name = 'Facebook'
    )
    GROUP BY ppl.id
) subquery;

### Задание №19


Составьте таблицу из полей:
 - name_of_fund — название фонда;
 - name_of_company — название компании;
 - amount — сумма инвестиций, которую привлекла компания в раунде.
 
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT
    f.name AS name_of_fund,
    c.name AS name_of_company,
    fr.raised_amount AS amount
FROM
    fund AS f
JOIN
    investment AS inv ON f.id = inv.fund_id
JOIN
    funding_round AS fr ON inv.funding_round_id = fr.id
JOIN
    company AS c ON inv.company_id = c.id
WHERE
    c.milestones > 6
    AND fr.funded_at BETWEEN '2012-01-01' AND '2013-12-31';

### Задание №20

Выгрузите таблицу, в которой будут такие поля:
 - название компании-покупателя;
 - сумма сделки;
 - название компании, которую купили;
 - сумма инвестиций, вложенных в купленную компанию;
 - доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
 
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT
    acquirer.name AS acquirer_company_name,
    acquisition.price_amount AS deal_amount,
    acquired.name AS acquired_company_name,
    acquired.funding_total AS invested_amount,
    ROUND(acquisition.price_amount / NULLIF(acquired.funding_total, 0)) AS investment_multiplier
FROM
    company AS acquirer
JOIN
    acquisition ON acquirer.id = acquisition.acquirer_id
JOIN
    company AS acquired ON acquisition.acquired_id = acquired.id
WHERE
    acquisition.price_amount > 0
    AND acquired.funding_total > 0
ORDER BY
    deal_amount DESC,
    acquired_company_name
LIMIT 10;

### Задание №21

Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT
    c.name AS company_name,
    --c.category_code,
   -- fr.funded_at,
    EXTRACT(MONTH FROM fr.funded_at) AS funding_month
FROM
    company AS c
JOIN
    funding_round AS fr ON c.id = fr.company_id
WHERE
    c.category_code = 'social'
    AND fr.funded_at >= '2010-01-01' AND fr.funded_at <= '2013-12-31'
    AND fr.raised_amount <> 0
ORDER BY
    c.name ASC, fr.funded_at ASC;

### Задание №22

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
 - номер месяца, в котором проходили раунды;
 - количество уникальных названий фондов из США, которые инвестировали в этом месяце;
 - количество компаний, купленных за этот месяц;
 - общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH fundings AS (
    SELECT
        EXTRACT(MONTH FROM CAST(fr.funded_at AS DATE)) AS funding_month,
        COUNT(DISTINCT CASE WHEN f.country_code = 'USA' THEN f.id END) AS us_funds
    FROM
        fund AS f
    LEFT JOIN
        investment AS i ON f.id = i.fund_id
    LEFT JOIN
        funding_round AS fr ON i.funding_round_id = fr.id
    WHERE
        EXTRACT(YEAR FROM CAST(fr.funded_at AS DATE)) BETWEEN 2010 AND 2013
    GROUP BY
        funding_month
),
acquisitions AS (
    SELECT
        EXTRACT(MONTH FROM CAST(acquired_at AS DATE)) AS funding_month,
        COUNT(acquired_company_id) AS bought_co,
        SUM(price_amount) AS sum_total
    FROM
        acquisition
    WHERE
        EXTRACT(YEAR FROM CAST(acquired_at AS DATE)) BETWEEN 2010 AND 2013
    GROUP BY
        funding_month
)
SELECT
    fnd.funding_month,
    COALESCE(fnd.us_funds, 0) AS us_funds,
    COALESCE(acq.bought_co, 0) AS bought_co,
    COALESCE(acq.sum_total, 0) AS sum_total
FROM
    fundings AS fnd
LEFT JOIN
    acquisitions AS acq ON fnd.funding_month = acq.funding_month;

### Задание №23

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в `2011`, `2012` и `2013` годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за `2011` год от большего к меньшему.

In [ ]:
WITH y_11 AS (
    SELECT 
        country_code AS country,
        AVG(funding_total) AS y_2011
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at::DATE) IN (2011, 2012, 2013)
    GROUP BY country, EXTRACT(YEAR FROM founded_at)
    HAVING EXTRACT(YEAR FROM founded_at) = '2011'
),

y_12 AS (
    SELECT 
        country_code AS country,
        AVG(funding_total) AS y_2012
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at::DATE) IN (2011, 2012, 2013)
    GROUP BY country, EXTRACT(YEAR FROM founded_at)
    HAVING EXTRACT(YEAR FROM founded_at) = '2012'
),

y_13 AS (
    SELECT 
        country_code AS country,
        AVG(funding_total) AS y_2013
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at::DATE) IN (2011, 2012, 2013)
    GROUP BY country, EXTRACT(YEAR FROM founded_at)
    HAVING EXTRACT(YEAR FROM founded_at) = '2013'
)

SELECT 
    y_11.country, 
    y_2011, 
    y_2012, 
    y_2013
FROM y_11
JOIN y_12 ON y_11.country = y_12.country
JOIN y_13 ON y_12.country = y_13.country
ORDER BY y_2011 DESC;